In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# The Complete Training Loop -- From Raw Text to a Trained Language Model -- Vizuara

> **What you will build:** A complete, end-to-end training pipeline that takes raw text and produces a language model that can generate coherent text. Every component from the previous notebooks comes together here.

## 1. Why Does This Matter?

This is the culmination of everything we have built. In Notebooks 1-4, we developed each component of the training pipeline individually:

1. **Tokenizer** -- converting text to numbers (BPE)
2. **Dataset/DataLoader** -- creating batched training samples
3. **Optimizer** -- updating weights intelligently (Adam)
4. **Scheduler + Clipping** -- stabilizing training

Now we assemble all of them into a single, working training loop and actually train a language model. By the end of this notebook, you will have a model that generates text -- trained entirely from scratch.

The training loop pattern we build here is the same one used by GPT, LLaMA, Mistral, and every modern LLM. Only the scale differs.

## 2. Building Intuition

The training loop is a cycle that repeats thousands of times:

1. **Tokenize** raw text once (preprocessing)
2. **Load** a batch of input-target pairs
3. **Forward pass** -- model predicts next tokens
4. **Compute loss** -- cross-entropy between predictions and targets
5. **Backward pass** -- compute gradients
6. **Clip** gradients to prevent explosions
7. **Update** learning rate schedule
8. **Step** the optimizer to update weights
9. **Repeat** from step 2

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import subprocess

subprocess.check_call(['pip', 'install', '-q', 'tiktoken'])
import tiktoken

print("All dependencies loaded!")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Let us visualize the training loop as a cycle.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

# Draw the training cycle
stages = [
    ('Load\nBatch', 0),
    ('Forward\nPass', 45),
    ('Compute\nLoss', 90),
    ('Backward\nPass', 135),
    ('Clip\nGradients', 180),
    ('Update\nLR', 225),
    ('Optimizer\nStep', 270),
    ('Log\nMetrics', 315),
]

radius = 3
for name, angle_deg in stages:
    angle = np.radians(90 - angle_deg)  # Start from top, go clockwise
    x = radius * np.cos(angle)
    y = radius * np.sin(angle)

    circle = plt.Circle((x, y), 0.7, facecolor='#e8f4f8', edgecolor='#2980b9',
                         linewidth=2, zorder=3)
    ax.add_patch(circle)
    ax.text(x, y, name, ha='center', va='center', fontsize=9, fontweight='bold', zorder=4)

# Draw arrows between stages
for i in range(len(stages)):
    angle1 = np.radians(90 - stages[i][1])
    angle2 = np.radians(90 - stages[(i + 1) % len(stages)][1])

    x1 = radius * np.cos(angle1)
    y1 = radius * np.sin(angle1)
    x2 = radius * np.cos(angle2)
    y2 = radius * np.sin(angle2)

    # Shorten arrow to not overlap circles
    dx = x2 - x1
    dy = y2 - y1
    length = np.sqrt(dx**2 + dy**2)
    shrink = 0.8 / length

    ax.annotate('', xy=(x2 - dx * shrink, y2 - dy * shrink),
                xytext=(x1 + dx * shrink, y1 + dy * shrink),
                arrowprops=dict(arrowstyle='->', color='#e74c3c', lw=2))

ax.text(0, 0, 'Training\nLoop', ha='center', va='center', fontsize=14,
        fontweight='bold', color='#e74c3c')

ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('The Training Loop Cycle', fontsize=15, fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig('training_cycle.png', dpi=150, bbox_inches='tight')
plt.show()
print("Checkpoint: Every iteration follows this exact cycle.")

## 3. The Mathematics

### Cross-Entropy Loss for Language Models

For a sequence of length $T$ with vocabulary size $V$, the model outputs logits $z \in \mathbb{R}^{B \times T \times V}$. The cross-entropy loss is:

$$\mathcal{L} = -\frac{1}{B \cdot T} \sum_{b=1}^{B} \sum_{t=1}^{T} \log \frac{e^{z_{b,t,y_{b,t}}}}{\sum_{v=1}^{V} e^{z_{b,t,v}}}$$

where $y_{b,t}$ is the target token at position $t$ in sample $b$.

### Perplexity

Perplexity measures how "surprised" the model is by the data:

$$\text{PPL} = e^{\mathcal{L}}$$

A perplexity of 100 means the model is, on average, as uncertain as if it were choosing uniformly from 100 tokens. Lower is better.

In [ ]:
# Demonstrate cross-entropy and perplexity
V = 50257  # GPT-2 vocab size

# Random model (untrained): loss should be ~log(V)
random_loss = math.log(V)
random_ppl = math.exp(random_loss)
print(f"Random model (V={V:,}):")
print(f"  Expected loss: ln({V:,}) = {random_loss:.2f}")
print(f"  Expected perplexity: {random_ppl:.0f}")

# Good model: much lower loss
good_loss = 3.0
good_ppl = math.exp(good_loss)
print(f"\nGood model:")
print(f"  Loss: {good_loss}")
print(f"  Perplexity: {good_ppl:.1f}")

# Great model:
great_loss = 2.0
great_ppl = math.exp(great_loss)
print(f"\nGreat model:")
print(f"  Loss: {great_loss}")
print(f"  Perplexity: {great_ppl:.1f}")

## 4. Let's Build It -- Component by Component

### 4.1 Data Pipeline

In [ ]:
class TextDataset(Dataset):
    """Language model dataset with sliding window."""

    def __init__(self, tokens, context_length):
        self.tokens = torch.tensor(tokens, dtype=torch.long)
        self.context_length = context_length

    def __len__(self):
        return len(self.tokens) - self.context_length

    def __getitem__(self, idx):
        x = self.tokens[idx : idx + self.context_length]
        y = self.tokens[idx + 1 : idx + self.context_length + 1]
        return x, y

# Prepare training data
raw_text = """
In the beginning, the universe was created. This made a lot of people very angry
and has been widely regarded as a bad move. Many were increasingly of the opinion
that they'd all made a big mistake in coming down from the trees in the first place.
And some said that even the trees had been a bad move, and that no one should ever
have left the oceans. The ships hung in the sky in much the same way that bricks don't.
Ford Prefect knew where his towel was. The answer to the ultimate question of life,
the universe, and everything was forty-two. Arthur Dent did not enjoy Thursdays.
He never could get the hang of Thursdays. The Vogons were not combative. They were
merely thorough. It was not for nothing that the phrase "as useful as a Vogon guard
in an ideological debate" was so widely used. The great thing about space was that
it was so incredibly, amazingly, mind-bogglingly big. Really big. You just won't believe
how vastly, hugely, mind-bogglingly big it is. The probability of navigation was remote.
""" * 100  # Repeat for more training data

enc = tiktoken.get_encoding("gpt2")
tokens = enc.encode(raw_text)
print(f"Training data: {len(raw_text):,} characters -> {len(tokens):,} tokens")
print(f"Compression ratio: {len(raw_text) / len(tokens):.1f} chars/token")

CONTEXT_LENGTH = 64
BATCH_SIZE = 16

# Split into train/val
split_idx = int(0.9 * len(tokens))
train_dataset = TextDataset(tokens[:split_idx], CONTEXT_LENGTH)
val_dataset = TextDataset(tokens[split_idx:], CONTEXT_LENGTH)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

print(f"\nTrain: {len(train_dataset):,} samples, {len(train_loader)} batches")
print(f"Val:   {len(val_dataset):,} samples, {len(val_loader)} batches")

### 4.2 Transformer Language Model

In [ ]:
class TransformerLM(nn.Module):
    """A small Transformer language model for training pipeline demonstration."""

    def __init__(self, vocab_size, d_model=128, n_heads=4, n_layers=4,
                 context_length=64, dropout=0.1):
        super().__init__()

        self.context_length = context_length

        # Token + positional embeddings
        self.token_emb = nn.Embedding(vocab_size, d_model)
        self.pos_emb = nn.Embedding(context_length, d_model)
        self.dropout = nn.Dropout(dropout)

        # Transformer layers
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=d_model * 4,
            dropout=dropout,
            batch_first=True,
            norm_first=True,
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)

        # Output head
        self.ln_f = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, vocab_size, bias=False)

        # Weight tying: share token embeddings with output head
        self.head.weight = self.token_emb.weight

        # Causal mask
        self.register_buffer('causal_mask',
            torch.triu(torch.ones(context_length, context_length), diagonal=1).bool()
        )

        self._init_weights()

    def _init_weights(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, x):
        B, T = x.shape
        pos = torch.arange(T, device=x.device).unsqueeze(0)

        tok_emb = self.token_emb(x)
        pos_emb = self.pos_emb(pos)
        x = self.dropout(tok_emb + pos_emb)

        x = self.transformer(x, mask=self.causal_mask[:T, :T], is_causal=True)
        x = self.ln_f(x)
        logits = self.head(x)

        return logits

# Create model
VOCAB_SIZE = enc.n_vocab
model = TransformerLM(
    vocab_size=VOCAB_SIZE,
    d_model=128,
    n_heads=4,
    n_layers=4,
    context_length=CONTEXT_LENGTH,
).to(device)

# Count parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"Model parameters: {num_params:,}")
print(f"Model size: ~{num_params * 4 / 1e6:.1f} MB (float32)")

# Test forward pass
test_input = torch.randint(0, VOCAB_SIZE, (2, CONTEXT_LENGTH)).to(device)
test_output = model(test_input)
print(f"\nForward pass test:")
print(f"  Input shape:  {test_input.shape}  (batch, seq_len)")
print(f"  Output shape: {test_output.shape}  (batch, seq_len, vocab)")

### 4.3 Training Infrastructure

In [ ]:
def get_lr(step, warmup_steps, total_steps, lr_max=3e-4, lr_min=1e-5):
    """Warmup + cosine decay learning rate schedule."""
    if step < warmup_steps:
        return lr_max * step / warmup_steps
    progress = (step - warmup_steps) / max(1, total_steps - warmup_steps)
    progress = min(progress, 1.0)
    return lr_min + 0.5 * (lr_max - lr_min) * (1 + math.cos(math.pi * progress))


@torch.no_grad()
def evaluate(model, val_loader, device):
    """Compute validation loss and perplexity."""
    model.eval()
    total_loss = 0
    num_batches = 0
    for x, y in val_loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        loss = F.cross_entropy(logits.view(-1, logits.size(-1)), y.view(-1))
        total_loss += loss.item()
        num_batches += 1
    avg_loss = total_loss / max(num_batches, 1)
    model.train()
    return avg_loss, math.exp(avg_loss)


@torch.no_grad()
def generate(model, enc, prompt, max_tokens=100, temperature=0.8, device='cpu'):
    """Generate text from a prompt."""
    model.eval()
    tokens = enc.encode(prompt)
    tokens = torch.tensor(tokens, dtype=torch.long).unsqueeze(0).to(device)

    for _ in range(max_tokens):
        # Crop to context length
        x = tokens[:, -model.context_length:]
        logits = model(x)
        logits = logits[:, -1, :] / temperature

        # Sample from the distribution
        probs = F.softmax(logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)
        tokens = torch.cat([tokens, next_token], dim=1)

    model.train()
    return enc.decode(tokens[0].tolist())

# Test generation before training (should be random garbage)
print("Generation BEFORE training:")
print(generate(model, enc, "The answer", max_tokens=50, device=device))

## 5. Your Turn

### TODO 1: Complete the Training Loop

Fill in the missing parts of the training loop.

In [ ]:
def train_model(model, train_loader, val_loader, epochs, device,
                lr_max=3e-4, warmup_fraction=0.1, max_grad_norm=1.0):
    """
    Complete training loop with all components.
    """
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr_max,
                                    betas=(0.9, 0.999), weight_decay=0.01)

    total_steps = len(train_loader) * epochs
    warmup_steps = int(warmup_fraction * total_steps)

    # Tracking
    train_losses = []
    val_losses = []
    learning_rates = []
    grad_norms = []
    step = 0

    print(f"Training for {epochs} epochs ({total_steps} steps, {warmup_steps} warmup)")
    print("=" * 70)

    for epoch in range(epochs):
        epoch_loss = 0
        epoch_start = time.time()

        for batch_x, batch_y in train_loader:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            # TODO 1a: Forward pass -- compute logits
            logits = model(batch_x)   # Shape: (B, T, V)

            # TODO 1b: Compute cross-entropy loss
            # Reshape logits to (B*T, V) and targets to (B*T)
            loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)),
                batch_y.view(-1)
            )

            # TODO 1c: Backward pass
            optimizer.zero_grad()
            loss.backward()

            # TODO 1d: Compute and record gradient norm, then clip
            total_norm = torch.nn.utils.clip_grad_norm_(
                model.parameters(), max_norm=max_grad_norm
            )
            grad_norms.append(total_norm.item())

            # TODO 1e: Update learning rate
            lr = get_lr(step, warmup_steps, total_steps, lr_max)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            learning_rates.append(lr)

            # TODO 1f: Step the optimizer
            optimizer.step()

            epoch_loss += loss.item()
            train_losses.append(loss.item())
            step += 1

        # End of epoch
        avg_train_loss = epoch_loss / len(train_loader)
        val_loss, val_ppl = evaluate(model, val_loader, device)
        val_losses.append(val_loss)
        elapsed = time.time() - epoch_start

        print(f"Epoch {epoch+1:>2d}/{epochs} | "
              f"Train Loss: {avg_train_loss:.4f} | "
              f"Val Loss: {val_loss:.4f} | "
              f"Val PPL: {val_ppl:.1f} | "
              f"LR: {lr:.2e} | "
              f"Time: {elapsed:.1f}s")

    return {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'learning_rates': learning_rates,
        'grad_norms': grad_norms,
    }

### TODO 2: Experiment with Hyperparameters

After training, try modifying these hyperparameters and observe the effect:

In [ ]:
# TODO: Try different configurations and compare results
# Configuration 1 (default): lr_max=3e-4, warmup=10%, clip=1.0
# Configuration 2: lr_max=1e-3, warmup=5%, clip=1.0 (higher lr)
# Configuration 3: lr_max=3e-4, warmup=10%, clip=0.5 (tighter clipping)
# Configuration 4: lr_max=3e-4, no warmup, clip=1.0 (skip warmup)

# Which configuration gives the lowest validation loss?
# Hypothesis: ________________
# Result: ________________

print("Experiment with hyperparameters after the main training run!")

## 6. Putting It All Together

Let us run the complete training loop.

In [ ]:
# Train the model!
metrics = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=15,
    device=device,
    lr_max=3e-4,
    warmup_fraction=0.1,
    max_grad_norm=1.0,
)

## 7. Training and Results

In [ ]:
# Visualize training results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Training loss
ax = axes[0, 0]
ax.plot(metrics['train_losses'], linewidth=0.5, alpha=0.3, color='#3498db')
# Smoothed
window = min(50, len(metrics['train_losses']) // 10)
if window > 1:
    smoothed = np.convolve(metrics['train_losses'], np.ones(window)/window, mode='valid')
    ax.plot(range(window-1, len(metrics['train_losses'])), smoothed, linewidth=2, color='#2c3e50')
ax.set_xlabel('Step')
ax.set_ylabel('Loss')
ax.set_title('Training Loss', fontweight='bold')
ax.grid(alpha=0.3)

# Validation loss
ax = axes[0, 1]
epochs_x = np.arange(1, len(metrics['val_losses']) + 1)
ax.plot(epochs_x, metrics['val_losses'], 'o-', linewidth=2, color='#e74c3c', markersize=6)
ax.set_xlabel('Epoch')
ax.set_ylabel('Validation Loss')
ax.set_title('Validation Loss', fontweight='bold')
ax.grid(alpha=0.3)

# Learning rate
ax = axes[1, 0]
ax.plot(metrics['learning_rates'], linewidth=2, color='#2ecc71')
ax.set_xlabel('Step')
ax.set_ylabel('Learning Rate')
ax.set_title('Learning Rate Schedule', fontweight='bold')
ax.grid(alpha=0.3)

# Gradient norms
ax = axes[1, 1]
ax.plot(metrics['grad_norms'], linewidth=0.5, alpha=0.5, color='#9b59b6')
ax.axhline(y=1.0, color='black', linestyle='--', linewidth=1, label='Clip threshold')
ax.set_xlabel('Step')
ax.set_ylabel('Gradient Norm')
ax.set_title('Gradient Norms', fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

plt.suptitle('Complete Training Dashboard', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.savefig('training_dashboard.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Final training loss: {metrics['train_losses'][-1]:.4f}")
print(f"Final validation loss: {metrics['val_losses'][-1]:.4f}")
print(f"Final perplexity: {math.exp(metrics['val_losses'][-1]):.1f}")

## 8. Final Output: Text Generation

In [ ]:
# Generate text with the trained model
prompts = [
    "The answer to",
    "In the beginning",
    "The ships hung",
    "Arthur Dent",
    "The probability",
]

print("=" * 60)
print("  TEXT GENERATION FROM TRAINED MODEL")
print("=" * 60)

for prompt in prompts:
    generated = generate(model, enc, prompt, max_tokens=80, temperature=0.8, device=device)
    print(f"\nPrompt: '{prompt}'")
    print(f"Output: {generated[:200]}")
    print("-" * 60)

In [ ]:
# Final summary: the complete pipeline
print("=" * 60)
print("  THE COMPLETE TRAINING PIPELINE")
print("=" * 60)
print()
print("  TOKENIZE  -->  Text to numbers (BPE)")
print("  BATCH     -->  Sliding window + DataLoader")
print("  FORWARD   -->  Model predicts next tokens")
print("  LOSS      -->  Cross-entropy with targets")
print("  BACKWARD  -->  Compute gradients")
print("  CLIP      -->  Cap gradient norms")
print("  SCHEDULE  -->  Warmup + cosine decay")
print("  STEP      -->  AdamW updates weights")
print()
print("  This pattern trains EVERY modern language model.")
print("  GPT, LLaMA, Mistral, Gemma, Phi -- same loop,")
print("  different scale.")
print("=" * 60)
print()
print(f"  Model size:    {sum(p.numel() for p in model.parameters()):,} parameters")
print(f"  Final loss:    {metrics['val_losses'][-1]:.4f}")
print(f"  Perplexity:    {math.exp(metrics['val_losses'][-1]):.1f}")
print(f"  Vocab size:    {VOCAB_SIZE:,}")
print(f"  Context:       {CONTEXT_LENGTH}")

## 9. Reflection and Next Steps

**What we built:** A complete, working training pipeline that takes raw text and produces a language model capable of generating text. Every component -- tokenizer, dataset, dataloader, optimizer, scheduler, gradient clipping -- works together.

**Key takeaways:**
- The training loop pattern is universal: tokenize, batch, forward, loss, backward, clip, schedule, step
- Cross-entropy loss + next-token prediction is how language models learn
- Perplexity = exp(loss) measures model quality
- All the components we built individually are essential -- remove any one and training breaks
- Scale is the main difference between our model and GPT-4: the same loop, but with trillions of tokens and billions of parameters

**The architecture provides the capacity to learn. The training pipeline provides the process of learning. Together, they produce intelligence.**